# Quantum State Tomography

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

A central challenge on the path towards large-scale quantum computing is the engineering of high-quality quantum state. A method that accurately and reliably characterize unknown quantum state $\rho$ is desirable.
Quantum state tomography is the process by which a quantum state is reconstructed using measurements on an ensemble of identical quantum states.

This tutorial introduces Quantum State Tomography (QST), covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).

## Theory
The general principle behind quantum state tomography is that by repeatedly performing many measurements on quantum systems described by identical density matrices, frequency counts can be used to infer probabilities, and these probabilities are combined to determine a density matrix which fits the best with the observations.


### Pauli transfer matrix representation
Manipulations with quantum maps can be conveniently carried out using the superoperator formalism. In this formalism density operators $\rho$ on a Hilbert space of dimension $d=2^n$ are represented as vectors $| \rho \rangle \rangle $ in Hilbert-Schmidt space of dimension $d^2$. The Hilbert-Schmidt inner product is defined as

$$
    \langle\langle A | B \rangle\rangle = \textrm{Tr} \left[ A^\dagger B \right],
$$

 where $A, \; B$ are linear operators.
Although superoperators can be defined relative to any basis of $\mathcal{H}_n$,  we continue to find it convenient to use the Pauli basis.
Single-qubit case, Pauli basis are:

$$
\begin{align}
    I = \left[\begin{matrix} 1 & 0 \\ 0 & 1 \end{matrix}\right], \;
    X = \left[\begin{matrix} 0 & 1 \\ 1 & 0 \end{matrix}\right], \;
    Y = \left[\begin{matrix} 0 & -i \\ i & 0 \end{matrix}\right], \;
    Z = \left[\begin{matrix} 1 & 0 \\ 0 & -1 \end{matrix}\right].
\end{align}
$$

From this point on, we use rescaled Pauli operators $P_k \to P_k / \sqrt{d}$. This way the basis is properly normalized, and we avoid having to write factors of $d$ everywhere.

$$
    P_k \in \left\{ I / \sqrt{2}, X / \sqrt{2},  Y / \sqrt{2},  Z / \sqrt{2} \right\}^{\otimes n}.
$$

We represent state $\rho$ as state vectors

$$
    | \rho \rangle \rangle = \sum_k |k\rangle\rangle \langle \langle k| \rho \rangle\rangle,\sum_k |k\rangle\rangle \langle \langle k| = I,
$$

with components

$$
    \langle\langle k | \rho \rangle\rangle = \textrm{Tr} \left[ P_k \rho \right].
$$

This is simply a restatement of the completeness of the Pauli basis:

$$
    \rho = \sum_k \textrm{Tr} \left[P_k \rho\right] P_k.
$$


### State tomography method

Quantum state tomography attempts to characterize an unknown state $\rho$ by measuring its components $| \rho \rangle \rangle$. To obtain these components, it is usually more experimentally convenient to use a different basis of measurement operators $\left\{ E_j \right\}_{j=1}^M (M \geq d^2)$ than span the Hilbert-Schmidt space $\mathscr{L}(\mathcal{H}_d)$, where $d = 2^n$ and $n$ is the number of qubits. Generally, one measures the $M$ expectation values (or you may say probabilities)

$$
    \forall j \in [M],\; p_j := \langle\langle E_j | \rho \rangle \rangle = \textrm{Tr} \left[ E_j \rho \right].
$$

Inserting the complete set of Pauli basis, we have

$$
    p_j = \langle\langle E_j | \rho \rangle\rangle = \langle\langle E_j |
    \left( \sum_{k=1}^{d^2} | k \rangle\rangle \langle\langle k | \right) | \rho \rangle\rangle
    = \sum_{k=1}^{d^2} \langle\langle E_j | k \rangle \rangle \langle \langle k | \rho \rangle \rangle,
$$

where $\langle\langle E_j | k \rangle\rangle = \textrm{Tr} \left[ P_k E_j \right]$, $\langle \langle k | \rho \rangle \rangle = \textrm{Tr} \left[ P_k \rho \right]$.
Since $\left\{ E_j \right\}_j$ is chosen in advance by the experimenter, we may assume the matrix $A$ is known whose elements are defined as

$$
    A_{jk} = \langle\langle E_j | k \rangle\rangle \Rightarrow
    A = \left[
         \begin{matrix}
           \langle\langle E_1 | 1 \rangle\rangle & \langle\langle E_1 | 2 \rangle\rangle & \dots & \langle\langle E_1 | d^2 \rangle\rangle \\
           \langle\langle E_2 | 1 \rangle\rangle & \langle\langle E_2 | 2 \rangle\rangle & \dots & \langle\langle E_2 | d^2 \rangle\rangle \\
           \vdots & \vdots & \ddots & \vdots \\
           \langle\langle E_M | 1 \rangle\rangle & \langle\langle E_M | 2 \rangle\rangle & \dots & \langle\langle E_M | d^2 \rangle\rangle \\
          \end{matrix}
      \right].
$$

The probability $p_j$ can be estimated as the sample average $ \hat{p}_j = \sum_{i=1}^N \hat{p}_{i|j} / N $ of $N$ single-shot measurements of $E_j$, where $\hat{p}_{i|j} = 0$ or $1$ is the outcome of the $i$-th measurement. The expected value of $\mathbb{E} \hat{p}_j = \sum_{i=1}^N \mathbb{E} \hat{p}_{i|j} / N = p_j$. The sample average $\hat{p}_j$ approaches the true value $p_j$ as $N \to \infty$.
Then we define the following $M \times 1$ dimensional real vector:

$$
    \hat{p} := \left[ \begin{matrix} \hat{p}_1 \\  \hat{p}_2 \\ \vdots \\ \hat{p}_M \\  \end{matrix}\right],
$$

which can be experimentally obtained, as shown in the figure below, we measure $\rho$ with each $E_j$ for $N$ times, and obtain the expectation values $\hat{p}_j$.

![QST](./figures/qst-circuit.png "Figure 1: We construct such circuits to estimate expectation values. ")
 
What's more, notice that

$$
    | \rho \rangle\rangle =
    \left[ \begin{matrix} \langle\langle 1 | \rho \rangle\rangle \\ \langle\langle 2 | \rho \rangle\rangle \\ \vdots \\ \langle\langle d^2 | \rho \rangle\rangle \\  \end{matrix} \right].
$$

The problem can be rewritten in matrix form as

$$
    \hat{p} = A | \rho \rangle \rangle,
$$

where both $A$ and $\hat{p}$ are known, and we aim to estimate the unknown $| \rho \rangle\rangle$.
If the set of measurement observables $\left\{ E_j \right\}_j$ are carefully chosen so that $A$ is invertible (in which case it must hold that $M = d^2$), the solution is

$$
    |\rho\rangle\rangle = A^{-1} \hat{p}.
$$

Actually, the optimization problem can be analytically resolved provided that the $d^2$ columns of the matrix $A$ are linearly independent:

$$
    | \rho \rangle\rangle = \left(A^\dagger A\right)^{-1} A^\dagger \hat{p},
$$

where $A^\dagger A$ is known as the Gram matrix and is always invertible.
Finally, the flow chart is as follows.
 
![QST](./figures/qst-chart.png "Figure 2: We complete QST according to this flow chart.")
 

## Practice

We demonstrate quantum state tomography on the Bell state.

First, we import the necessary libraries.

In [1]:
from QCompute import *
import QCompute
from qcompute_qep.tomography import StateTomography
from qcompute_qep.quantum.metrics import state_fidelity
from qcompute_qep.utils.circuit import circuit_to_state

Then we set up the quantum program for preparing the Bell state.

In [2]:
qp = QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(2)
H(qp.Q[0])
CX(qp.Q[0], qp.Q[1])


 
![QST](./figures/qst-Bell-example.png "Figure 3: We take the Bell state as the target of QST. ")
 
The circuit is shown in the figure above.
Before QST, we compute numerically the ideal target state for reference.

In [ ]:
ideal_state = circuit_to_state(qp, vector=False)
print("ideal state operator...")
print(ideal_state)

Then we set the quantum computer (instance of QComputer). The QuantumComputer can be a simulator or a hardware interface. The rest is simple, we initialize a StateTomography instance, call the tomography procedure and obtain the noisy quantum state, we set the method as 'inverse' means that we estimate $| \rho \rangle\rangle$ with $|\rho\rangle\rangle = A^{-1}\hat{p}$.

In [ ]:
# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"

qc = QCompute.BackendName.CloudBaiduQPUQian

# Initialize a StateTomography instance
st = StateTomography()
# Call the tomography procedure and obtain the noisy quantum state
noisy_state = st.fit(qp, qc, method='inverse', shots=8192)

In quantum mechanics, notably in quantum information theory, fidelity is a measure fo the "closeness" of two quantum states. Since we can use fidelity to evaluate the effect of QST. Because we use local simulator, our estimate is very close to the ideal state.

In [ ]:
print('Fidelity between the ideal and noisy states: F = {:.5f}'.format(st.fidelity))

Fidelity between the ideal and noisy states: F = 0.80682


## Summary

This tutorial describes how to use Quantum State Tomography method to reconstruct the unknown quantum state on [Baidu Quantum Platform](https://quantum.baidu.com/).

## References
[1] Greenbaum, Daniel. "Introduction to quantum gate set tomography." [arXiv](https://arxiv.org/abs/1509.02921) preprint arXiv:1509.02921 (2015).
